# Ingest PBT News Releases

In [1]:
from tabula import read_pdf

In [2]:
%run configs.ipynb

In [3]:
#parameters cell
news_file_name = 'dec 2021 news, oct 2021 prod.pdf'

In [4]:
# Parameters
news_file_name = "sept 2021 news, jul 2021 prod.pdf"


In [5]:
pbt_news_DF = read_pdf(f'{raw_data_folder}/pbt_news/2021/{news_file_name}')[0]

pbt_news_DF

'pages' argument isn't specified.Will extract only from page 1 by default.


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,Underlying Properties,Net to Trust Sales,NaN,NaN
1,NaN,Volumes,Volumes,Average,Price
2,NaN,Oil Gas,Oil Gas,Oil,Gas
3,NaN,(bbls) (Mcf),(bbls) (Mcf),(per bbl),(per Mcf)
4,Current Month,NaN,NaN,NaN,NaN
5,Waddell Ranch,"102,735 249,359","77,051 187,019*",$71.32,$3.45**
6,Texas Royalties,"17,690 11,870","15,772 10,570*",$71.11,$8.09**
7,NaN,NaN,NaN,NaN,NaN
8,Prior Month,NaN,NaN,NaN,NaN
9,Waddell Ranch,"90,258 220,120","67,694 165,090*",$70.22,$4.21**


In [6]:
cols_pdf = 'Property', 'Gross Oil Volume (PBT News)', 'Net Oil Volume (PBT News)', 'Average Oil Price', 'Average Gas Price'
pbt_news_DF.columns = cols_pdf

In [7]:
pbt_news_DF.dropna(inplace = True)
pbt_news_DF

,Property,Gross Oil Volume (PBT News),Net Oil Volume (PBT News),Average Oil Price,Average Gas Price
5,Waddell Ranch,"102,735 249,359","77,051 187,019*",$71.32,$3.45**
6,Texas Royalties,"17,690 11,870","15,772 10,570*",$71.11,$8.09**
9,Waddell Ranch,"90,258 220,120","67,694 165,090*",$70.22,$4.21**
10,Texas Royalties,"17,820 9,894","15,854 8,797*",$69.06,$7.89**


### Splitting Net Oil Column to Get Gas Volume
### Applying Text Cleaning to remove commas

In [8]:
pbt_news_DF['Gross Gas Volume (PBT News)'] = pbt_news_DF['Gross Oil Volume (PBT News)'].apply(lambda x: int(x.split(' ')[1].replace(',', '')))

pbt_news_DF['Gross Oil Volume (PBT News)'] = pbt_news_DF['Gross Oil Volume (PBT News)'].apply(lambda x: int(x.split(' ')[0].replace(',', '')))

In [9]:
pbt_news_DF

,Property,Gross Oil Volume (PBT News),Net Oil Volume (PBT News),Average Oil Price,Average Gas Price,Gross Gas Volume (PBT News)
5,Waddell Ranch,102735,"77,051 187,019*",$71.32,$3.45**,249359
6,Texas Royalties,17690,"15,772 10,570*",$71.11,$8.09**,11870
9,Waddell Ranch,90258,"67,694 165,090*",$70.22,$4.21**,220120
10,Texas Royalties,17820,"15,854 8,797*",$69.06,$7.89**,9894


### Splitting Net Oil Column to Get Gas Volume
### Applying Text Cleaning to remove commas and asterisks

In [10]:
pbt_news_DF['Net Gas Volume (PBT News)'] = pbt_news_DF['Net Oil Volume (PBT News)'].apply(lambda x: int(x.split(' ')[1]\
                                                                                  .replace(',', '')\
                                                                                 .replace('*','')))

pbt_news_DF['Net Oil Volume (PBT News)'] = pbt_news_DF['Net Oil Volume (PBT News)'].apply(lambda x: int(x.split(' ')[0].replace(',', '')))

### Cleaning Prices Columns by removing asterisks and Dollar Sign

In [11]:
pbt_news_DF['Average Gas Price'] = pbt_news_DF['Average Gas Price'].apply(lambda x: x.replace('$','').replace('**',''))

pbt_news_DF['Average Oil Price'] = pbt_news_DF['Average Oil Price'].apply(lambda x: x.replace('$','').replace('**',''))

In [12]:
#filtering table to the 'current month' values
pbt_news_DF = pbt_news_DF.head(2)

In [13]:
pbt_news_DF

,Property,Gross Oil Volume (PBT News),Net Oil Volume (PBT News),Average Oil Price,Average Gas Price,Gross Gas Volume (PBT News),Net Gas Volume (PBT News)
5,Waddell Ranch,102735,77051,71.32,3.45,249359,187019
6,Texas Royalties,17690,15772,71.11,8.09,11870,10570


In [14]:
date_from_filename = news_file_name.split(',')[1]\
                                    .split('.')[0]\
                                    .strip()\
                                    .replace(' prod', '')

In [15]:
pbt_news_DF.insert(0, 'Date', date_from_filename.title())

In [16]:
pbt_news_DF

,Date,Property,Gross Oil Volume (PBT News),Net Oil Volume (PBT News),Average Oil Price,Average Gas Price,Gross Gas Volume (PBT News),Net Gas Volume (PBT News)
5,Jul 2021,Waddell Ranch,102735,77051,71.32,3.45,249359,187019
6,Jul 2021,Texas Royalties,17690,15772,71.11,8.09,11870,10570


# Write Table to File

In [17]:
to_file_name = news_file_name.split('.')[0] + '.csv'

to_file_name

'sept 2021 news, jul 2021 prod.csv'

In [18]:
pbt_news_DF.to_csv(f'{processed_data_folder}/pbt_news_prod/{to_file_name}', index = False)